# 0. Import libraries and packages

In [1]:
import pandas as pd
import os

# 1. Datasets - Load and Exploration

## 1.1 Fact table - Casinodaily 

In [2]:
casinodaily = pd.read_csv('../datasets/casinodaily.csv')

In [3]:
casinodaily.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 65790 entries, 0 to 65789
Data columns (total 9 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   UserID                65790 non-null  object 
 1   CountryId             65790 non-null  int64  
 2   CurrencyId            65790 non-null  int64  
 3   Date                  65790 non-null  object 
 4   CasinoGameId          65790 non-null  int64  
 5   CasinoProviderId      65790 non-null  int64  
 6   CasinoManufacturerId  65790 non-null  int64  
 7   GGR                   65790 non-null  float64
 8   Returns               65790 non-null  float64
dtypes: float64(2), int64(5), object(2)
memory usage: 4.5+ MB


In [4]:
casinodaily.head(20)

,UserID,CountryId,CurrencyId,Date,CasinoGameId,CasinoProviderId,CasinoManufacturerId,GGR,Returns
0,bo0Ykz7Nn7,106,15,2025-03-10,149671,110,101,32.411080,17.330133
1,1LbgZSf9Pz,106,15,2025-03-10,1192,21,2,70.529403,20.141626
2,icpWDuiljz,106,15,2025-03-10,17450,150,42,1.022822,1.716777
3,cJKkhthQ67,106,15,2025-03-10,15893,110,61,-25.228250,1035.080020
4,bs1jduSrAT,106,15,2025-03-10,166141,21,133,6.115386,16.613069
5,gNUus9HboE,106,15,2025-03-10,16521,21,151,-10.978345,125.273963
6,3Q6PtY0nOx,106,15,2025-03-10,149673,21,96,2.374548,8.007151
7,Xlod9GzDI2,106,15,2025-03-10,164960,150,42,1.528384,0.069696
8,HD8fEgVnPP,106,15,2025-03-10,164960,150,42,0.026586,0.437634
9,IFkJGFyMTi,106,15,2025-03-10,149725,21,120,1.132348,2.540670


In [5]:
casinodaily.CurrencyId.unique()

array([15, 10], dtype=int64)

In [6]:
casinodaily[casinodaily['CurrencyId']==15]

,UserID,CountryId,CurrencyId,Date,CasinoGameId,CasinoProviderId,CasinoManufacturerId,GGR,Returns
0,bo0Ykz7Nn7,106,15,2025-03-10,149671,110,101,32.411080,17.330133
1,1LbgZSf9Pz,106,15,2025-03-10,1192,21,2,70.529403,20.141626
2,icpWDuiljz,106,15,2025-03-10,17450,150,42,1.022822,1.716777
3,cJKkhthQ67,106,15,2025-03-10,15893,110,61,-25.228250,1035.080020
4,bs1jduSrAT,106,15,2025-03-10,166141,21,133,6.115386,16.613069
...,...,...,...,...,...,...,...,...,...
63566,VSzrt1FUb2,106,15,2025-05-27,17443,150,42,34.991952,93.855464
63567,gzSt6tXEAi,106,15,2025-05-27,167552,70,6,32.996778,10.477743
63568,wW92PLLw6Q,106,15,2025-05-27,17384,150,42,7.930727,3.850724
63569,KHBFoG9uDv,106,15,2025-05-27,167552,70,6,-18.192960,471.635336


In [7]:
casinodaily[casinodaily['CurrencyId']==15].count()

UserID                  51835
CountryId               51835
CurrencyId              51835
Date                    51835
CasinoGameId            51835
CasinoProviderId        51835
CasinoManufacturerId    51835
GGR                     51835
Returns                 51835
dtype: int64

In [8]:
casinodaily[casinodaily['CurrencyId']==10].CountryId.unique()

array([199], dtype=int64)

## 1.2 Dimension table - Users

In [9]:
users = pd.read_csv('../datasets/users.csv')

In [10]:
users.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7734 entries, 0 to 7733
Data columns (total 7 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   UserID     7734 non-null   int64 
 1   user_id    7734 non-null   object
 2   BirthDate  7734 non-null   object
 3   Sex        7734 non-null   object
 4   VIPStatus  7734 non-null   object
 5   Country    7734 non-null   object
 6   CountryID  7734 non-null   int64 
dtypes: int64(2), object(5)
memory usage: 423.1+ KB


In [11]:
users.head()

,UserID,user_id,BirthDate,Sex,VIPStatus,Country,CountryID
0,5762754,vFFv24LBfv,1989-05-02,F,Potential,NZD,106
1,5778624,uoqSXzqFy1,1991-10-04,F,NOT vip,NZD,106
2,5610434,3lZ5PE5f7F,1994-09-06,F,NOT vip,NZD,106
3,4523919,JPikNnu1o7,1991-10-11,M,Bronz E,NZD,106
4,4564697,pIrSKY0nZu,1991-03-06,M,Bronz E,NZD,106


In [12]:
users.Country.unique()

array(['NZD'], dtype=object)

### Convert BirthDate datatype to date

In [13]:
users['BirthDate'] =  pd.to_datetime(users['BirthDate'], errors = 'coerce')

In [14]:
users.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7734 entries, 0 to 7733
Data columns (total 7 columns):
 #   Column     Non-Null Count  Dtype         
---  ------     --------------  -----         
 0   UserID     7734 non-null   int64         
 1   user_id    7734 non-null   object        
 2   BirthDate  7733 non-null   datetime64[ns]
 3   Sex        7734 non-null   object        
 4   VIPStatus  7734 non-null   object        
 5   Country    7734 non-null   object        
 6   CountryID  7734 non-null   int64         
dtypes: datetime64[ns](1), int64(2), object(4)
memory usage: 423.1+ KB


## 1.3 Dimension table - Casino Manufacturers

In [15]:
manufacturers = pd.read_csv('../datasets/casinomanufacturers.csv', sep = ', "', engine = 'python')

In [16]:
manufacturers.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 319 entries, 0 to 318
Data columns (total 1 columns):
 #   Column                                                                                    Non-Null Count  Dtype 
---  ------                                                                                    --------------  ----- 
 0   "CasinoManufacturerId,""CasinoManufacturerName"",""FromDate"",""ToDate"",""LatestFlag"""  319 non-null    object
dtypes: object(1)
memory usage: 2.6+ KB


In [17]:
manufacturers.head(30)

,"""CasinoManufacturerId,""""CasinoManufacturerName"""",""""FromDate"""",""""ToDate"""",""""LatestFlag"""""""
0,"1,Viaden,2019-11-15 09:11:32.287,,1"
1,"2,MicroGaming,2019-11-15 09:11:32.287,2024-04-..."
2,"2,Micro Gaming,2024-04-16 06:11:35.627,,1"
3,"3,Evolution,2020-06-16 09:41:29.923,,1"
4,"4,ScientificGames,2019-11-15 09:11:32.287,2024..."
5,"4,Scientific Games,2024-04-11 06:51:11.840,,1"
6,"5,NetEnt,2019-11-15 09:11:32.287,,1"
7,"6,PlayTech,2019-11-15 09:11:32.287,,1"
8,"7,PlayNGo,2019-11-15 09:11:32.287,,1"
9,"8,Amatic,2019-11-15 09:11:32.287,,1"


### Split/spread data into multiple columns

In [18]:
manufacturers['"CasinoManufacturerId,""CasinoManufacturerName"",""FromDate"",""ToDate"",""LatestFlag"""'].str.split(',',n=4,expand=True).set_axis(['CasinoManufacturerId','CasinoManufacturerName','FromDate', 'ToDate', 'LatestFlag'],axis=1)

,CasinoManufacturerId,CasinoManufacturerName,FromDate,ToDate,LatestFlag
0,1,Viaden,2019-11-15 09:11:32.287,,1
1,2,MicroGaming,2019-11-15 09:11:32.287,2024-04-16 06:11:35.627,0
2,2,Micro Gaming,2024-04-16 06:11:35.627,,1
3,3,Evolution,2020-06-16 09:41:29.923,,1
4,4,ScientificGames,2019-11-15 09:11:32.287,2024-04-11 06:51:11.840,0
...,...,...,...,...,...
314,252,Relax Gaming,2025-05-06 09:07:57.960,,1
315,253,TrustyGaming,2025-05-07 13:30:55.363,,1
316,254,KitsuneStudios,2025-05-07 13:32:11.323,,1
317,255,AceRoll,2025-05-12 13:29:28.310,,1


In [19]:
manufacturers = manufacturers['"CasinoManufacturerId,""CasinoManufacturerName"",""FromDate"",""ToDate"",""LatestFlag"""'].str.split(',',n=4,expand=True).set_axis(['CasinoManufacturerId','CasinoManufacturerName','FromDate', 'ToDate', 'LatestFlag'],axis=1)

### Remove all " from column values

In [20]:
manufacturers['CasinoManufacturerId'] = manufacturers['CasinoManufacturerId'].str.replace('"', '')
manufacturers['CasinoManufacturerName'] = manufacturers['CasinoManufacturerName'].str.replace('"', '')
manufacturers['LatestFlag'] = manufacturers['LatestFlag'].str.replace('"', '')

### Convert CasinoManufacturerId and LatestFlag columns to integer

In [21]:
manufacturers[['CasinoManufacturerId', 'LatestFlag']] = manufacturers[['CasinoManufacturerId', 'LatestFlag']].apply(pd.to_numeric)

In [22]:
manufacturers[manufacturers['LatestFlag'] == 1]

,CasinoManufacturerId,CasinoManufacturerName,FromDate,ToDate,LatestFlag
0,1,Viaden,2019-11-15 09:11:32.287,,1
2,2,Micro Gaming,2024-04-16 06:11:35.627,,1
3,3,Evolution,2020-06-16 09:41:29.923,,1
5,4,Scientific Games,2024-04-11 06:51:11.840,,1
6,5,NetEnt,2019-11-15 09:11:32.287,,1
...,...,...,...,...,...
314,252,Relax Gaming,2025-05-06 09:07:57.960,,1
315,253,TrustyGaming,2025-05-07 13:30:55.363,,1
316,254,KitsuneStudios,2025-05-07 13:32:11.323,,1
317,255,AceRoll,2025-05-12 13:29:28.310,,1


In [23]:
manufacturers.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 319 entries, 0 to 318
Data columns (total 5 columns):
 #   Column                  Non-Null Count  Dtype 
---  ------                  --------------  ----- 
 0   CasinoManufacturerId    319 non-null    int64 
 1   CasinoManufacturerName  319 non-null    object
 2   FromDate                319 non-null    object
 3   ToDate                  319 non-null    object
 4   LatestFlag              319 non-null    int64 
dtypes: int64(2), object(3)
memory usage: 12.6+ KB


### Keeping only the latest manufacturers names

In [24]:
manufacturers = manufacturers[manufacturers['LatestFlag'] == 1]

## 1.4 Dimension table - Casino Providers

In [25]:
providers = pd.read_csv('../datasets/casinoproviders.csv')

In [26]:
providers.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 38 entries, 0 to 37
Data columns (total 2 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   CasinoProviderId    38 non-null     int64 
 1   CasinoProviderName  38 non-null     object
dtypes: int64(1), object(1)
memory usage: 736.0+ bytes


In [27]:
providers.head(50)

,CasinoProviderId,CasinoProviderName
0,15,Viaden
1,21,MicroGaming
2,21,GamesGlobal
3,30,Evolution
4,40,ScientificGames
5,60,NetEnt
6,70,PlayTech
7,80,Iforium
8,100,Edict
9,110,Nyx


### Dropping 4 rows. 2 of them have same id and similar name (EGTGaming, ESAGaming), the other 2 have same id and different name.

In [28]:
providers.drop([2, 10, 25, 29], inplace= True)

## 1.5 Dimension table - Currency Rates

In [29]:
currencyrates = pd.read_csv('../datasets/currencyrates.csv')

In [30]:
currencyrates.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 147 entries, 0 to 146
Data columns (total 5 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Date               147 non-null    object 
 1   FromCurrencyId     147 non-null    int64  
 2   ToCurrencyId       147 non-null    int64  
 3   ToCurrencySysname  147 non-null    object 
 4   EuroRate           147 non-null    float64
dtypes: float64(1), int64(2), object(2)
memory usage: 5.9+ KB


In [31]:
currencyrates

,Date,FromCurrencyId,ToCurrencyId,ToCurrencySysname,EuroRate
0,2025-01-03,2,15,NZD,0.544484
1,2025-01-02,2,15,NZD,0.543419
2,2025-01-01,2,15,NZD,0.539607
3,2025-01-06,2,15,NZD,0.544840
4,2025-01-05,2,15,NZD,0.544484
...,...,...,...,...,...
142,2025-05-24,2,15,NZD,0.525984
143,2025-05-23,2,15,NZD,0.525984
144,2025-05-22,2,15,NZD,0.522275
145,2025-05-21,2,15,NZD,0.524466


In [32]:
currencyrates.ToCurrencyId.unique()

array([15], dtype=int64)

# 3. Joined data table

In [33]:
casinodaily = casinodaily [casinodaily['CurrencyId']==15]

I have excluded data with CurrencyId = 10 since the Currency Rate table has only currency rate for CurrencyId = 15.

In [34]:
final_table = casinodaily \
    .merge(users, how = 'left', left_on = 'UserID', right_on= 'user_id')\
    .merge(manufacturers, how = 'left', on = 'CasinoManufacturerId')\
    .merge(providers, how = 'left', on = 'CasinoProviderId')\
    .merge(currencyrates, how = 'left', left_on =['CurrencyId', 'Date'], right_on = ['ToCurrencyId', 'Date'])

In [35]:
final_table.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 51835 entries, 0 to 51834
Data columns (total 25 columns):
 #   Column                  Non-Null Count  Dtype         
---  ------                  --------------  -----         
 0   UserID_x                51835 non-null  object        
 1   CountryId               51835 non-null  int64         
 2   CurrencyId              51835 non-null  int64         
 3   Date                    51835 non-null  object        
 4   CasinoGameId            51835 non-null  int64         
 5   CasinoProviderId        51835 non-null  int64         
 6   CasinoManufacturerId    51835 non-null  int64         
 7   GGR                     51835 non-null  float64       
 8   Returns                 51835 non-null  float64       
 9   UserID_y                51835 non-null  int64         
 10  user_id                 51835 non-null  object        
 11  BirthDate               51835 non-null  datetime64[ns]
 12  Sex                     51835 non-null  object

### Create Age column to create later AgeGroup column  
Instead of now any particular date can be used to do the following calculations

In [36]:
now = pd.to_datetime('now')

In [37]:
final_table['Age']=(now - final_table['BirthDate']).dt.total_seconds() / (60*60*24*365.25)

### Create AgeGroup column

In [38]:
final_table['AgeGroup'] = pd.cut(final_table['Age'], 
                            [0, 18, 21, 27, 33, 41, 51, 150], 
                            labels=['Under 18', '18-20', '21-26', '27-32', '33-40', '41-50', '50+'])

### Currency conversions

In [39]:
final_table['GGR_EUR'] = final_table['GGR']*final_table['EuroRate']
final_table['Returns_EUR'] = final_table['Returns']*final_table['EuroRate']

In [40]:
final_table.columns

Index(['UserID_x', 'CountryId', 'CurrencyId', 'Date', 'CasinoGameId',
       'CasinoProviderId', 'CasinoManufacturerId', 'GGR', 'Returns',
       'UserID_y', 'user_id', 'BirthDate', 'Sex', 'VIPStatus', 'Country',
       'CountryID', 'CasinoManufacturerName', 'FromDate', 'ToDate',
       'LatestFlag', 'CasinoProviderName', 'FromCurrencyId', 'ToCurrencyId',
       'ToCurrencySysname', 'EuroRate', 'Age', 'AgeGroup', 'GGR_EUR',
       'Returns_EUR'],
      dtype='object')

In [41]:
final_table = final_table[['Date', 'Country', 'Sex', 'AgeGroup','VIPStatus', 'CasinoManufacturerName', 'CasinoProviderName', 'GGR', 'Returns', 'GGR_EUR', 'Returns_EUR']]

In [42]:
final_table.head(50)

,Date,Country,Sex,AgeGroup,VIPStatus,CasinoManufacturerName,CasinoProviderName,GGR,Returns,GGR_EUR,Returns_EUR
0,2025-03-10,NZD,M,27-32,Bronz E,Hacksaw,Nyx,32.411080,17.330133,17.133307,9.161142
1,2025-03-10,NZD,F,33-40,POTENTIAL,Micro Gaming,MicroGaming,70.529403,20.141626,37.283606,10.647367
2,2025-03-10,NZD,F,41-50,POTENTIAL,PragmaticPlay,PragmaticPlay,1.022822,1.716777,0.540689,0.907531
3,2025-03-10,NZD,M,21-26,Bronz E,Blue Print,Nyx,-25.228250,1035.080020,-13.336284,547.169176
4,2025-03-10,NZD,F,33-40,Potential,INOGames,MicroGaming,6.115386,16.613069,3.232746,8.782083
5,2025-03-10,NZD,F,33-40,Potential,Triple Edge Studios,MicroGaming,-10.978345,125.273963,-5.803428,66.222949
6,2025-03-10,NZD,F,21-26,NOT vip,FortuneFactorStudios,MicroGaming,2.374548,8.007151,1.255245,4.232780
7,2025-03-10,NZD,M,27-32,Bronz E,PragmaticPlay,PragmaticPlay,1.528384,0.069696,0.807942,0.036843
8,2025-03-10,NZD,F,33-40,NOT vip,PragmaticPlay,PragmaticPlay,0.026586,0.437634,0.014054,0.231344
9,2025-03-10,NZD,F,41-50,POTENTIAL,Aurum,MicroGaming,1.132348,2.540670,0.598587,1.343062


In [43]:
final_table.count()

Date                      51835
Country                   51835
Sex                       51835
AgeGroup                  51835
VIPStatus                 51835
CasinoManufacturerName    51835
CasinoProviderName        51835
GGR                       51835
Returns                   51835
GGR_EUR                   51835
Returns_EUR               51835
dtype: int64

# 4. Aggregated Data

In [44]:
agr = final_table.groupby(['Date', 'Country', 'Sex', 'AgeGroup', 'VIPStatus', 'CasinoManufacturerName', 'CasinoProviderName'], as_index = False)[['GGR', 'Returns', 'GGR_EUR', 'Returns_EUR']].agg('sum')

C:\Users\anagn\AppData\Local\Temp\ipykernel_26696\2008910199.py:1: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  agr = final_table.groupby(['Date', 'Country', 'Sex', 'AgeGroup', 'VIPStatus', 'CasinoManufacturerName', 'CasinoProviderName'], as_index = False)[['GGR', 'Returns', 'GGR_EUR', 'Returns_EUR']].agg('sum')


In [45]:
agr = agr[agr['Returns']!=0].reset_index()

In [46]:
agr.columns

Index(['index', 'Date', 'Country', 'Sex', 'AgeGroup', 'VIPStatus',
       'CasinoManufacturerName', 'CasinoProviderName', 'GGR', 'Returns',
       'GGR_EUR', 'Returns_EUR'],
      dtype='object')

In [47]:
agr.drop(['index'], axis=1, inplace= True)

In [48]:
agr.columns

Index(['Date', 'Country', 'Sex', 'AgeGroup', 'VIPStatus',
       'CasinoManufacturerName', 'CasinoProviderName', 'GGR', 'Returns',
       'GGR_EUR', 'Returns_EUR'],
      dtype='object')

In [49]:
agr

,Date,Country,Sex,AgeGroup,VIPStatus,CasinoManufacturerName,CasinoProviderName,GGR,Returns,GGR_EUR,Returns_EUR
0,2025-01-01,NZD,F,18-20,NOT vip,PragmaticPlay,PragmaticPlay,0.100860,0.838061,0.054425,0.452224
1,2025-01-01,NZD,F,18-20,NOT vip,Stormcraft Studios,MicroGaming,2.351018,12.510926,1.268626,6.750986
2,2025-01-01,NZD,F,21-26,NOT vip,PragmaticPlay,PragmaticPlay,8.150890,4.655054,4.398279,2.511900
3,2025-01-01,NZD,F,21-26,NOT vip,Stormcraft Studios,MicroGaming,6.156527,18.786865,3.322106,10.137528
4,2025-01-01,NZD,F,27-32,NOT vip,1X2Gaming,Nyx,6.457596,0.037730,3.484565,0.020359
...,...,...,...,...,...,...,...,...,...,...,...
22368,2025-05-27,NZD,M,33-40,ELi t e,InspiredGaming,InspiredGaming,1.626519,0.581623,0.859410,0.307314
22369,2025-05-27,NZD,M,33-40,ELi t e,Micro Gaming,MicroGaming,-14.088332,104.423903,-7.443904,55.174844
22370,2025-05-27,NZD,M,33-40,ELi t e,PlayTech,PlayTech,21.461531,187.348387,11.339708,98.989960
22371,2025-05-27,NZD,M,50+,ELi t e,Evolution,EvolutionDirect,3.294783,0.380852,1.740877,0.201232


### Create target subdirectory

In [50]:
os.makedirs('../target', exist_ok=True)  

### Write data to csv file in target subdirectory

In [51]:
agr.to_csv('../target/aggregated_data.csv') 